In [17]:
output_dir = "/data/nst"

In [19]:
import sqltables
from pathlib import Path
import csv

In [3]:
db = sqltables.sqlite3.Database("/data/nst/annotations.sqlite3")

In [4]:
annotations = db.open_table("annotations")

In [5]:
annotations

|prefix|recording|audio\_file|path|text|speaker\_id|
|-|-|-|-|-|-|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512142\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512142\.wav\'|\'Alt er lagt til rette\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512261\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512261\.wav\'|\'uddannet\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512205\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512205\.wav\'|\'Formanden for rådet glæder sig over\,\\\\Komma at der er taget hul på problemerne med generationsskifte\.\\\\Punktum\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512244\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512244\.wav\'|\'Herning\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512126\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512126\.wav\'|\'elleve tusind og seksogtyve\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512296\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512296\.wav\'|\'liste\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512301\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512301\.wav\'|\'telefon privat\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512304\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512304\.wav\'|\'optag notat\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512163\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512163\.wav\'|\'Det betyder\, at når solen opvarmer væsken i solfangeren\, opstår der en forskel i massefylde mellem væsken i solfangeren og i beholderen\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512271\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512271\.wav\'|\'forklare\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512149\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512149\.wav\'|\'Det er de traditionelle druer\, der fortrinsvis anvendes\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512084\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512084\.wav\'|\'Det stigende behov for kraftvarme skyldes at tilslutningen til det kollektive varmesystem er øget betydeligt og fortsat vil øges\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512176\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512176\.wav\'|\'De fleste af aktiviteterne har ikke gennem de senere år genereret en indtjening\, der bare tilnærmelsesvis har kunnet modsvare omkostningerne\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512291\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512291\.wav\'|\'hastighed\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512186\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512186\.wav\'|\'Det gik slag i slag\.\'|\'509\'|
|\'train\/Stasjon6\/060700\'|\'r5650512\'|\'u0512065\.wav\'|\'\/data\/nst\/train\/Stasjon6\/060700\/adb\_0565\/speech\/scr0565\/06\/05650606\/r5650512\/u0512065\.wav\'|\'Han var den første\, der skaffede diasfilm til landet efter krigen\, og hans forretning udviklede sig til en stor\'|\'509\'|
|...|...|...|...|...|...|


In [13]:
import random
random.seed(123)
rows = list(annotations.view("select * from _ where prefix like 'train%' order by path"))
random.shuffle(rows)

In [14]:
val_frac = 0.1
test_frac = 0.1
val_idx = int(len(rows)*val_frac)
test_idx = val_idx + int(len(rows)*test_frac)
val_rows = rows[0:val_idx]
test_rows = rows[val_idx:test_idx]
train_rows = rows[test_idx:]

In [22]:
output_files = {
    "train.csv": train_rows,
    "dev.csv": val_rows,
    "test.csv": test_rows
}

In [23]:
for fname, data in output_files.items():
    with open(Path(output_dir) / fname, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["ID", "duration", "wav", "spk_id", "wrd"])
        for row in data:
            row_id = f"{row.prefix}-{row.recording}-{row.audio_file}"
            writer.writerow([row_id, "5", row.path, row.speaker_id, row.text])